# Catapult codesign

Model: `(13,21,2), n_filters=5, pool_size=3`

## Setup

Please follow [these instructions](https://github.com/GiuseppeDiGuglielmo/catapult_venvs) to setup an environment for Catapult AI NN (_hls4ml_).

You can either use the _hls4ml_ release with Catapult (2024.2) or point to Siemens or official GitHub repository. [Here we provide some details.](https://github.com/GiuseppeDiGuglielmo/catapult_compare_releases)

In [ ]:
# Path to the Catapult installation directory
!echo $MGC_HOME

In [ ]:
# Path to the hls4ml installation directory
# It can be point to either the Catapult installation directory or a copy of Siemens/Official GitHub repo 
!echo $PYTHONPATH

## Import libraries

Disable some console warnings on the ASIC-group servers

In [ ]:
!rm -rf myproject_hls4ml_prj*

In [ ]:
import os

In [ ]:
import hls4ml

import numpy as np
import math
import yaml
import json
# from tensorflow.keras.models import Model
from models import CreateModel

from matplotlib import colors

from loss import *
from qkeras import quantized_bits

import matplotlib.pyplot as plt

## Set parameters

In [ ]:
# Enable batch normalization in the model (True, False)
batch_norm_enabled = False
# True = use all of the time slices, False = use a subset of the timeslices
timeslices_all_enable = False

batch_size = 1000

In [ ]:
# See: https://docs.google.com/document/d/1ZoqVyJOOAXhzt2egMWh3OoNJ6lWq5lNR6sjcYON4Vlo/edit?tab=t.0#heading=h.k6tyal7z5t5l
dataset_name = "dataset_2s"
# 50x12.5x100 micron pixel sensor => 13x21 pixel sensor array
sensor_geometry_name = "50x12P5x100"
# Either 20 or 2 timeslices
timeslices_name = "timeslices20" if timeslices_all_enable else "timeslices2"
timeslices_val = -1 if timeslices_all_enable else [0, 19]
#
batch_size_name = f"bs{batch_size}"

In [ ]:
# You can define a JSON configuration file locally
# {
#    "data_base_dir": "/data/dajiang/smartPixels",
#    "tfrecords_base_dir" : "/data/dajiang/smartPixels",
#    "model_base_dir": "/home/dajiang/smart-pixels-ml/weights"
# }
config_file_path = 'config.json'

# If the file does not exist, the notebook uses default values for those entries
data_base_dir = "/data/dajiang/smartPixels/dataset_2s"
tfrecords_base_dir = "/data/dajiang/smartPixels/tfrecords"
npy_base_dir = "/data/dajiang/smartPixels/npy"
model_base_dir = "/home/dajiang/smart-pixels-ml/weights"

if os.path.exists(config_file_path):
    with open(config_file_path, 'r') as file:
        data = json.load(file)
        data_base_dir = data.get('data_base_dir')
        tfrecords_base_dir = data.get('tfrecords_base_dir')
        npy_base_dir = data.get('npy_base_dir')
        model_base_dir = data.get('model_base_dir')
    print(f"Use config info from file: {data_base_dir}, {tfrecords_base_dir}, {npy_base_dir}, {model_base_dir}")
else:
    print(f"File does not exist. Use default config info: {data_base_dir}, {tfrecords_base_dir}, {npy_base_dir}, {model_base_dir}")

In [ ]:
model_name = f"{dataset_name}_{sensor_geometry_name}_{timeslices_name}_{batch_size_name}" + ("_bnorm" if batch_norm_enabled else "")

best_model_weights_hdf5 = f"{model_base_dir}/weights_7pitches/best_model_{model_name}_weights.hdf5"
best_model_architecture_json = f"{model_base_dir}/weights_7pitches/best_model_{model_name}_architecture.json"
best_model_hdf5 = f"{model_base_dir}/weights_7pitches/best_model_{model_name}.hdf5"

In [ ]:
# TODO: Set the right precision
FXD_W = 4 # Fixed-point precision, word bit width
FXD_I = 1 # Fixed-point precision, integer-part bit width

## Load data

In [ ]:
batch_size = 100

X = np.load(f'{npy_base_dir}/X_{timeslices_name}_val.npy')[:batch_size,]
y = np.load(f'{npy_base_dir}/y_{timeslices_name}_val.npy')[:batch_size,]

### Visualize data

In [ ]:
def plot_event(X, index):
    if index < 0 or index >= X.shape[0]:
        print("Index out of range. Please provide a valid index.")
        return

    slices_num = X[index].shape[2]
    
    fig, axes = plt.subplots(1, slices_num, figsize=(8, 4))

    divnorm = colors.TwoSlopeNorm(vmin=-1., vcenter=0., vmax=8.5)
    
    img1 = axes[0].imshow(X[index, :, :, 0], interpolation='nearest', origin='lower', cmap='bwr', norm=divnorm)
    axes[0].set_title(f'Event: {index} - Slice 1')
    axes[0].set_xticks(range(0, X.shape[2], max(1, X.shape[2] // 5)))
    axes[0].set_yticks(range(0, X.shape[1], max(1, X.shape[1] // 5)))

    img2 = axes[1].imshow(X[index, :, :, 1], interpolation='nearest', origin='lower', cmap='bwr', norm=divnorm)
    axes[1].set_title(f'Event: {index} - Slice 2')
    axes[1].set_xticks(range(0, X.shape[2], max(1, X.shape[2] // 5)))
    axes[1].set_yticks(range(0, X.shape[1], max(1, X.shape[1] // 5)))

    plt.show()

# Plot some images
for i in range(2):
    plot_event(X, i)

### Quantize data

## QKeras model

### Load QKeras model

In [ ]:
# Load the whole model from HDF5 file
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

co = {"custom_loss": custom_loss}
_add_supported_quantized_objects(co)
model = load_model(best_model_hdf5, custom_objects=co)
model.summary()

In [ ]:
# TODO: This gives an error:
# ValueError: Layer count mismatch when loading weights from file. Model expected 5 layers, found 9 saved layers.
#
# # Load model weights from HDF5 file, while recreate architecture from scripts
# model = CreateModel((13,21,2), n_filters=5, pool_size=3)
# model.load_weights(best_model_weights_hdf5)
# model.summary()

### Slice QKeras model

In [ ]:
# SLICE_TO_LAYER_NUM = 12
# print("Layer count: {}/{}".format(SLICE_TO_LAYER_NUM, len(model.layers)-1))
# assert(SLICE_TO_LAYER_NUM < len(model.layers))
# model = Model(inputs=model.input, 
#                   outputs=model.layers[SLICE_TO_LAYER_NUM].output)

In [ ]:
# model.summary()

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=True)

### Run QKeras model

#### Prediction

In [ ]:
y_qkeras = model.predict(np.ascontiguousarray(X))

#### Profiling

In [ ]:
qkeras_trace = hls4ml.model.profiling.get_ymodel_keras(model, X)

#### Save .dat files

In [ ]:
# Save input features and model predictions just the top 20
np.savetxt("tb_input_features.dat", X.reshape(batch_size, -1), fmt="%f")
np.savetxt("tb_output_predictions.dat", y_qkeras, fmt="%f")
#np.savetxt("y_test_labels.dat", y_test, fmt="%d")

## hls4ml model

### Configure hls4ml model

In [ ]:
config_ccs = hls4ml.utils.config.create_config(
    backend = "Catapult",
    project_name = "myproject",
    output_dir = "myproject_hls4ml_prj",
    tech = "asic",
    asiclibs = "saed32rvt_tt0p78v125c_beh",
    asicfifo = "hls4ml_lib.mgc_pipe_mem",
    clock_period = 10,
    #io_type = "io_stream",
    io_type = "io_parallel",
    csim=0, SCVerify=0, Synth=1
)
#print(config_ccs)

In [ ]:
config_ccs["HLSConfig"] = hls4ml.utils.config_from_keras_model(
    model,
    granularity="name",
    default_precision="ac_fixed<16,6,true>",
    default_reuse_factor=1
)

# NOTE: fine tune average-pooling-2d
# TODO: Can this be fixed directly in the average-pooling-2d implementation?
config_ccs["HLSConfig"]['LayerName']['average_pooling2d']['Precision']['result'] = 'fixed<12,1>'

# Point to the model definition, weights/biase values and C++ testbench data files
config_ccs["KerasH5"] = best_model_weights_hdf5
config_ccs["KerasJson"] = best_model_architecture_json
config_ccs["InputData"] = "tb_input_features.dat"
config_ccs["OutputPredictions"] = "tb_output_predictions.dat"

In [ ]:
# TODO: is this necessary?
with open("myproject_config.yml", "w") as yaml_file:
    yaml.dump(config_ccs, yaml_file, explicit_start=False, default_flow_style=False)

In [ ]:
# Enable tracing for all of the layers
for layer in config_ccs["HLSConfig"]["LayerName"].keys():
    print("Enable tracing for layer:", layer)
    config_ccs["HLSConfig"]["LayerName"][layer]["Trace"] = True

In [ ]:
# Convert QKeras model to Catapult HLS C++
hls_model_ccs = hls4ml.converters.keras_to_hls(config_ccs)

In [ ]:
hls4ml.utils.plot_model(hls_model_ccs, show_shapes=True, show_precision=True, to_file="qkeras.png")
hls4ml.utils.plot_model(hls_model_ccs, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
# Writing HLS project
hls_model_ccs.compile()

### Run hls4ml model

#### Prediction

In [ ]:
y_ccs = hls_model_ccs.predict(np.ascontiguousarray(X))

#### Profiling

In [ ]:
# Run tracing on the test set for the hls4ml model (fixed-point precision) 
pred_ccs, trace_ccs = hls_model_ccs.trace(X)

## Compare QKeras and hls4ml

#### Trace visual inspection

In [ ]:
# Print the traces on console
N_ELEMENTS = 16

# Backup print options
bkp_threshold = np.get_printoptions()["threshold"]
bkp_linewidth = np.get_printoptions()["linewidth"]

# Set print options
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

# print(trace_ccs.keys())
# print(qkeras_trace.keys())
for key in trace_ccs.keys():
    if key == "q_separable_conv2d_depthwise":
        continue
    print("-------")
    print(key.replace("_pointwise", ""), trace_ccs[key].shape)
    print("[keras] ", key.replace("_pointwise", ""), qkeras_trace[key.replace("_pointwise", "")].flatten()[:N_ELEMENTS])
    print("[hls4ml]", key.replace("_pointwise", ""), trace_ccs[key].flatten()[:N_ELEMENTS])
    
# Restore print options
np.set_printoptions(threshold=bkp_threshold, linewidth=bkp_linewidth)

#### MSE per layer

In [ ]:
def mse(actual, predicted):
    return ((actual - predicted) ** 2).mean()

for key in trace_ccs.keys():
    if key == "q_separable_conv2d_depthwise":
        continue
    print("-------")
    print("MSE {} {}".format(key.replace("_pointwise", ""), mse(trace_ccs[key].flatten(), qkeras_trace[key.replace("_pointwise", "")].flatten())))

#### Parity plots

We use [parity plots](https://en.wikipedia.org/wiki/Parity_plot) to compare _hls4ml_ and QKeras implementations of each layer.
- If the _hls4ml_ implementation is accurate (i.e. matches QKeras implementation), the ouput values should lie close to the diagonal line `y=x`, indicating equivalence.
- Deviations from this line highlight inaccuracies or implementation errors.

In [ ]:
layers = [layer for layer in trace_ccs.keys() if "_alpha" not in layer and layer != "q_separable_conv2d_depthwise"]
num_layers = len(layers)

num_columns = 2
num_rows = math.ceil(num_layers / num_columns)

fig, axes = plt.subplots(num_rows, num_columns, figsize=(12, 6 * num_rows))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Loop through layers and plot
for i, layer in enumerate(layers):
    klayer = layer.replace("_pointwise", "").replace("_linear", "")

    min_x = min(np.amin(trace_ccs[layer]), np.amin(qkeras_trace[klayer]))
    max_x = max(np.amax(trace_ccs[layer]), np.amax(qkeras_trace[klayer]))

    x = trace_ccs[layer].flatten()
    y = qkeras_trace[klayer].flatten()

    golden_min_x = np.amin(qkeras_trace[klayer])
    golden_max_x = np.amax(qkeras_trace[klayer])
    range_size = abs(golden_min_x) + abs(golden_max_x)
    integer_bits = math.ceil(math.log2(range_size))
    min_abs_value = np.min(np.abs(qkeras_trace[klayer][qkeras_trace[klayer] != 0]))
    if min_abs_value > 0:
        decimal_bits = abs(math.floor(math.log2(min_abs_value)))
    else:
        decimal_bits = 0  # If no non-zero value exists
    mse_value = np.mean((x - y) ** 2)

    ax = axes[i]
    ax.plot([min_x, max_x], [min_x, max_x], color="gray", linestyle="--", label="Diagonal")
    scatter = ax.scatter(x, y, s=0.2, color="red", label="Data Points")
    
    stats_text = (
        f"Range: [{golden_min_x:.2f}, {golden_max_x:.2f}]\n"
        f"Range Size: {range_size:.2f}\n"
        f"Bits (I): {integer_bits}\n"
        f"Bits (D): {decimal_bits}\n"
        f"MSE: {mse_value:.2e}"
    )
    ax.text(
        0.05, 0.95, stats_text,
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white")
    )

    ax.set_xlabel(f"hls4ml {layer}")
    ax.set_ylabel(f"QKeras {klayer}")
    ax.set_title(f"Comparison QKeras vs. hls4ml ({klayer})")
    ax.grid(True)

# Remove empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout to prevent overlap
plt.tight_layout()

plt.savefig("comparison_qkeras_hls4m.jpeg")
plt.show()


## Model synthesis

In [ ]:
%%time

# TODO: Check the parameters for the build function (Catapult)

#report = hls_model.build(csim=True, synth=False, cosim=False, validation=False, export=False, vsynth=False, reset=False)
report = hls_model_ccs.build(csim=True, synth=True, cosim=False, validation=False, export=False, vsynth=False, bup=False)

### Results